In [4]:
import numpy as np

In [5]:
#Policy. When we are in state 0, we choose action 0, and when we are in state 1, we choose action 1.
pi = {0:0,1:1}

In [6]:
#Dynamics
p = {}
#p[s,a]={(r,s'):P(r,s'|s,a)}
p[(0,1)]={(-10,0):1}
p[(1,1)]={(-10,0):1}
p[(1,0)]={(-1,1):1}
p[(0,0)]={(0,0):0.95,(0,1):0.05}

POLICY EVALUATION

Computing the state-value function for a given policy

In [10]:
def policy_evaluation(policy,threshold,gamma):
    #initialize value function as zeros
    V = {i:0 for i in range(len(policy))}
    
    difference_large = True
    while difference_large:
        #current value function becomes the old value function
        V_old = V.copy()
        difference_large = False
        
        #loop over states
        for s in range(len(policy)):
            V[s]=0
            
            #choose action based on policy and state
            a = policy[s]
            
            for reward_and_next_state,prob in p[(s,a)].items():
                reward = reward_and_next_state[0]
                next_state = reward_and_next_state[1]
                
                V[s] += prob * (reward + gamma*(V_old[next_state]))
            difference = np.abs(V[s] - V_old[s])
            difference_large = difference_large or difference>threshold
    return V

In [11]:
V = policy_evaluation(pi,0.01,gamma=0.9)
V

{0: -4.219324840306157, 1: -13.78870283073678}

The new value function tells us that being in state 0 and then following $\pi$ gives an expected discounted return of -4.219324840306157, while being in state 1 and then following $\pi$ gives an expected discounted return of -13.78870283073678 

POLICY IMPROVEMENT

Changes the policy based on the value function. In every state, we look for the action that gives the highest value

In [31]:
def policy_improvement(V,gamma,n_actions):
    Q = {}
    new_policy = {}
    
    #loop over states
    for state in range(len(V)):
        best_value = -np.inf
        
        #loop over actions
        for action in range(n_actions):
            Q[state,action] = 0
            
            #loop over possible transitions with rewards
            for reward_and_next_state,prob in p[(state,action)].items():
                reward = reward_and_next_state[0]
                next_state = reward_and_next_state[1]
                
                #compute value of taking action in state
                Q[state,action] += prob * (reward + gamma*V[next_state])
            
            #if the new state-action value is higher
            if Q[state,action] > best_value:
                best_value = Q[state,action]
                best_action = action
        
        #assign best action to state
        new_policy[state] = best_action
    
    return new_policy, Q

In [32]:
new_pi = policy_improvement(V,0.9,n_actions=2)
new_pi

({0: 0, 1: 0},
 {(0, 0): -3.095875788387243,
  (0, 1): -12.785539542518226,
  (1, 0): -9.992264459898548,
  (1, 1): -12.785539542518226})

POLICY ITERATION

Iterating policy evaluation and policy improvement until stable

In [37]:
def policy_iteration(policy,gamma,n_actions):
    
    while True:
        V = policy_evaluation(policy=policy,threshold=0.001,gamma=gamma)
        
        old_policy = policy
        policy,Q = policy_improvement(V,gamma=gamma,n_actions=n_actions)
        
        policy_stable = True
        
        for state in range(len(V)):
            if policy[state] != old_policy[state]:
                policy_stable = False
                
        if policy_stable == True:
            return policy, Q

In [38]:
policy, Q = policy_iteration(pi,0.9,2)

In [39]:
policy

{0: 0, 1: 0}

In [40]:
Q

{(0, 0): -3.095875788387243,
 (0, 1): -12.785539542518226,
 (1, 0): -9.992264459898548,
 (1, 1): -12.785539542518226}